<a href="https://colab.research.google.com/github/sanjaynagi/AgamPrimer/blob/main/Primer-Design-in-Anopheles-gambiae-dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# First, install some packages we require
!pip install AgamPrimer==0.3.4 primer3-py malariagen_data -q 

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 397 kB 5.2 MB/s 
     |████████████████████████████████| 71 kB 343 kB/s 
     |████████████████████████████████| 2.3 MB 52.3 MB/s 
     |████████████████████████████████| 185 kB 57.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.6 MB/s 
     |████████████████████████████████| 5.7 MB 60.3 MB/s 
     |████████████████████████████████| 301 kB 76.3 MB/s 
     |████████████████████████████████| 123 kB 73.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.8 MB/s 
     |████████████████████████████████| 245 kB 52.3 MB/s 
     |████████████████████████████████| 1.6 MB 58.7 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 


In [1]:
# Import libraries 
import AgamPrimer
import pandas as pd
import malariagen_data
import numpy as np
import primer3
import matplotlib.pyplot as plt

##**AgamPrimer: Primer design considering genetic variation in *Anopheles gambiae***
**Author**: [Sanjay Curtis Nagi](https://sanjaynagi.github.io/)    
**Email**: sanjay.nagi@lstmed.ac.uk  

Often, we would like to design primers for PCR applications, such as genotyping (PCR, TaqMan, LNA) or gene expression qPCR (SYBR, TaqMan, LNA). However, single nucleotide polymorphisms (SNPs) in primer binding sites can result in differences or failures in PCR amplification, referred to as null alleles. 

In general, mismatches caused by SNPs are more of a problem as you move towards the 3' end. I recommend reading a really good article on this topic on the IDT website - [Consider SNPs when designing PCR and qPCR assays](https://eu.idtdna.com/pages/education/decoded/article/considering-snps-when-designing-pcr-and-qpcr-assays). In *An. gambiae s.l*, there is a [huge amount of genetic variation](https://genome.cshlp.org/content/30/10/1533.full); a SNP found approximately every 1.9 bases (!), which makes considering SNPs even more important when designing molecular assays. Thanks to primer3-py and the fantastic malariagen_data package, we can do all of this in the cloud, hosted by google!  

\

####**Google Colab**

If you are unfamiliar with iPython notebooks and google colab, I encourage you to read the [FAQ](https://research.google.com/colaboratory/faq.html) and watch the following [introduction](https://www.youtube.com/watch?v=inN8seMm7UI). In general, the cells contain python code, and can be run by pressing the play button next to each cell, and should be run in order.

You may want to save a copy of the notebook into your google drive (`file -> save a copy in Drive`).

In [81]:
#@title **Selecting Primer Parameters** { run: "auto" }
#@markdown In the below cells, replace the values with those desired for your primers.

assay_type = 'qPCR primers'           #@param ["gDNA primers", "gDNA primers + probe", "probe", "qPCR primers"]
assay_name = 'coeae1f'        #@param {type:"string"}
min_amplicon_size = 60        #@param {type:"integer"}
max_amplicon_size = 120      #@param {type:"integer"}
amplicon_size_range = [[min_amplicon_size, max_amplicon_size]]
n_primer_pairs = 6            #@param {type:"slider", min:1, max:20, step:1}

#@markdown    
#@markdown target_loc is required for gDNA primers and probes, and transcript required for qPCR primers.
contig =  "2L"                #@param ['2L', '2R', '3L', '3R', 'X']

target_loc =  '0'                #@param {type:"string"}
target_loc = int(target_loc)
transcript =  'AGAP006227-RA'         #@param {type:"string"} 


if any(item in assay_type for item in ['gDNA', 'probe']):
  assert target_loc > 0, "Target location must be above 0 and less than the contig length"
elif assay_type == 'qPCR primers':
  assert len(transcript) > 2, "Transcript ID is not valid, should be vectorbase ID such as 'AGAP004707-RD'"

Load sequence data for chromosomal arm of choice, using the [malariagen_data API](https://malariagen.github.io/vector-data/ag3/api.html):

In [82]:
# Connect to the malariagen_data ag3 API
ag3 = malariagen_data.Ag3() #pre=True
genome_seq = ag3.genome_sequence(region=contig)
print(f"Our genome sequence for {contig} is {genome_seq.shape[0]} bp long")

Our genome sequence for 2L is 49364325 bp long


Now we need to extract the bit of sequence we need. We will use functions in the [AgamPrimer](https://pypi.org/project/AgamPrimer/) package.

In [83]:
if any(item in assay_type for item in ['gDNA', 'probe']):
  # genomic DNA
  target_sequence, gdna_pos, seq_parameters = AgamPrimer.prepare_gDNA_sequence(target_loc=target_loc, amplicon_size_range=amplicon_size_range, genome_seq=genome_seq, assay_name=assay_name, assay_type=assay_type)
elif assay_type == 'qPCR primers':
  # quantitative PCR 
  target_sequence, exon_junctions, gdna_pos, seq_parameters = AgamPrimer.prepare_cDNA_sequence(transcript=transcript, gff=ag3.geneset(), genome_seq=genome_seq, assay_name=assay_name)

DEBUG:malariagen_data.ag3:geneset: handle region


Exon junctions for AGAP006227-RA: [ 375  629  973 1364 1514 1651] [28545771, 28546096, 28546517, 28546975, 28547196, 28547556] 



Now we have our target sequence. Lets take a look...

In [84]:
target_sequence

'AATTGTTTAAGTTTCTTGTAAATTTATCTTTAAAGCAACCAACAACGACGTACATGAAAAATCTAATAATTTGATTTATCAATTTAATAAAAAATATAACTACGATATTTAGTAAGCTACTAAAGATGCAATCAAACTGTACTACCAGCACCGTATAAAGTGTTTACCTCACTTAATACAACTGTTCCCGTTTGTACAACGAATCCCACAGCAGCATGCGTCTCATTTCCGGCAAATCGATAAAGCTCACACCGCCATTGTTAATATTTAAACACTTGTACGGCTCCACCTTTGCACCGACCGGTTGCCATATATCGCCAACACTGCCGCCAACATTCGGGCTGCCCACGGTCGCAAAGTTAGTAAACAACTCGACCAGCGTTTGCATCGTCCTGTACTCCATCGTGCCCACATCCGGCACCGGGCTGAACACGTTCTTAAACAGATACGAAAGATCATCCGCATGGGCGGTACCGCGCACGTTACGATCGCAAAAATAGATGCGATAGTGATTGTACGTCTCCGAATCCACTGCAAACCGGTAGACGAACGTTTTGGCCGGCTTTTGGGAGCTGATCCTCGAGCAGACGGTACGGTGCAGCCCGTGCCAGAAGAGCTTATCGGTCATGAGCGTTAAATAGCCTTCCCGATTGTCGAACGATGGTTGGGATCCACCGTAGTAGAACTTTTTCAACTTCAGCCCATACTGCTGGCATGCTTGCGATCCACGCACCAGCTCTAGCTCGGTAGGTACGAGATACTCGAAGTTTTTCAGATTATCCATTATGGCGGGGCTTTCCTTGATGGAGCTCAAGCAGAACAATCCTTCCTCGGCGTTTCCCCCGATCAGTATGTCGATGTCGTTGCTCCATGCCTCACGGCACATTTCTAATGGTGCCTTCGGAATGAAAGTGCTTTCTGTTACGTACGGTTCCACCACTGGACCAAAAGCGAACAGAATACGGTTTTCCAGCTCATTCTTAGTTAGTAGCAGATCCT

We need to set up some python dictionaries containing our sequence and primer parameters, this will be our input to primer3. In the below cell, you can modify or add primer3 parameters, such as optimal TM, GC content etc etc. A full list of possible parameters and their functions can be found in the [primer3 manual](https://primer3.org/manual.html).

In [85]:
primer_parameters  =  {
        'PRIMER_NUM_RETURN':n_primer_pairs,
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_TASK':'generic',
        'PRIMER_MIN_SIZE': 17,
        'PRIMER_MAX_SIZE': 24,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 30.0,
        'PRIMER_MAX_GC': 75.0,
        'PRIMER_PRODUCT_SIZE_RANGE': amplicon_size_range,
        'PRIMER_MIN_THREE_PRIME_DISTANCE':3,          # this parameter is the minimum distance between successive pairs. If 1, it means successive primer pairs could be identical bar one base shift
        'PRIMER_INTERNAL_OPT_SIZE': 16,               # Probe size preferences if selected, otherwise ignored
        'PRIMER_INTERNAL_MIN_SIZE': 10,
        'PRIMER_INTERNAL_MAX_SIZE': 22,
        'PRIMER_INTERNAL_MIN_TM': 45,
        'PRIMER_INTERNAL_MAX_TM':65,                # Probe considerations are quite relaxed, assumed that LNAs will be used later to affect TM
        # Extra primer3 parameters can go here
        # In the same format as above                       
    }

primer_parameters = AgamPrimer.primer_params(primer_parameters, assay_type) ## adds some parameters depending on assay type

#### **Run the primer3 algorithm!**

In [86]:
primer_dict = primer3.designPrimers(seq_args=seq_parameters, global_args=primer_parameters)

It should be *fast*. The output, which we call 'primer_dict', is a python dictionary containing the full results from primer3. We will turn this into a pandas dataframe (i.e a useful, pretty table), containing just the necessary bits of information. First, we'll print some information from the primer3 run.

In [87]:
AgamPrimer.primer3_run_statistics(primer_dict, assay_type)

PRIMER_LEFT_EXPLAIN  :  considered 15144, GC content failed 1124, low tm 4710, high tm 4177, high hairpin stability 11, ok 5122 

PRIMER_RIGHT_EXPLAIN  :  considered 15144, GC content failed 931, low tm 4764, high tm 4238, high hairpin stability 3, ok 5208 

PRIMER_PAIR_EXPLAIN  :  considered 216846, unacceptable product size 6304, no overlap of required point 210531, primer in pair overlaps a primer in a better pair 5442, ok 11 

PRIMER_LEFT_NUM_RETURNED  :  6 

PRIMER_RIGHT_NUM_RETURNED  :  6 

PRIMER_INTERNAL_NUM_RETURNED  :  0 

PRIMER_PAIR_NUM_RETURNED  :  6 



Now lets wrangle this into an easy to look at table.

In [88]:
primer_df = AgamPrimer.primer3_to_pandas(primer_dict, assay_type)
primer_df

7


primer_pair,0,1,2,3,4,5
parameter,,,,,,
PRIMER_LEFT_SEQUENCE,AGTCCGGTCCGTACATTTCG,CACTTGTACGGCTCCACCTT,AGCTTATCGGTCATGAGCGT,AACAACTCGACCAGCGTTTG,TTGCCGGTTTGAGCTCTTTC,CATTTCGGTTCCGCTTGAGC
PRIMER_RIGHT_SEQUENCE,CACGAAAGAGCTCAAACCGG,CAAACGCTGGTCGAGTTGTT,GGTGGATCCCAACCATCGTT,TAAGAACGTGTTCAGCCCGG,ACAAGATCGTGGGCAGTGAG,GAAAGAGCTCAAACCGGCAA
PRIMER_LEFT_TM,59.828097,59.965104,59.252745,59.344267,59.048442,60.179189
PRIMER_RIGHT_TM,59.487048,59.344267,60.034477,60.320073,60.036952,59.048442
PRIMER_LEFT_GC_PERCENT,55.0,55.0,50.0,50.0,50.0,55.0
PRIMER_RIGHT_GC_PERCENT,55.0,50.0,55.0,55.0,55.0,50.0
PRIMER_LEFT,"(1425, 20)","(272, 20)","(614, 20)","(365, 20)","(1506, 20)","(1438, 20)"
PRIMER_RIGHT,"(1528, 20)","(384, 20)","(675, 20)","(439, 20)","(1571, 20)","(1525, 20)"
PRIMER_PAIR_PRODUCT_SIZE,104,113,62,75,66,88



We can write this to .tsv and excel files, which can be explored in other editors. To download a file from colab to your local computer, click the folder panel on the left-hand sidebar, the three dots next your primers.tsv/.xlsx file, and download.

In [89]:
primer_df.to_csv(f"{assay_name}.{assay_type}.primers.tsv", sep="\t")
primer_df.to_excel(f"{assay_name}.{assay_type}.primers.xlsx")

##**Looking for variation using the ag1000g resource and malariagen API**

In Ag3, samples are organised into sample sets. We can load any sample set from the Ag3 resource, but there are quite a few! Lets look at what each sample set contains, breaking it down by species, year and country. 

In [90]:
metadata = ag3.sample_metadata()

Load sample metadata:   0%|          | 0/28 [00:00<?, ?it/s]

In [91]:
pivot_country_year_taxon = (
    metadata
    .pivot_table(
        index=["sample_set", "year", "country"], 
        columns=["taxon"], 
        values="sample_id",
        aggfunc="count",
        fill_value=0
    )
)

pivot_country_year_taxon

taxon                                                arabiensis  coluzzii  \
sample_set   year  country                                                  
AG1000G-AO    2009 Angola                                     0        81   
AG1000G-BF-A  2012 Burkina Faso                               0        82   
AG1000G-BF-B  2014 Burkina Faso                               3        53   
AG1000G-BF-C  2004 Burkina Faso                               0         0   
AG1000G-CD    2015 Democratic Republic of the Congo           0         0   
AG1000G-CF    1993 Central African Republic                   0         5   
              1994 Central African Republic                   0        13   
AG1000G-CI    2012 Cote d'Ivoire                              0        80   
AG1000G-CM-A  2009 Cameroon                                   0         0   
AG1000G-CM-B  2005 Cameroon                                   0         7   
AG1000G-CM-C  2013 Cameroon                                   2        19   
AG1000G-FR    2011 Mayotte                                    0         0   
AG1000G-GA-A  2000 Gabon                                      0         0   
AG1000G-GH    2012 Ghana                                      0        64   
AG1000G-GM-A  2011 Gambia, The                                0         0   
AG1000G-GM-B  2012 Gambia, The                                0         2   
AG1000G-GM-C  2012 Gambia, The                                0        12   
AG1000G-GN-A  2012 Guinea                                     0         4   
AG1000G-GN-B  2012 Guinea                                     0         7   
                   Mali                                       0        28   
AG1000G-GQ    2002 Equatorial Guinea                          0         0   
AG1000G-GW    2010 Guinea-Bissau                              0         0   
AG1000G-KE    2000 Kenya                                      0         0   
              2007 Kenya                                      3         0   
              2012 Kenya                                     10         0   
AG1000G-ML-A  2014 Mali                                       0        27   
AG1000G-ML-B  2004 Mali                                       2        36   
AG1000G-MW    2015 Malawi                                    41         0   
AG1000G-MZ    2003 Mozambique                                 0         0   
              2004 Mozambique                                 0         0   
AG1000G-TZ    2012 Tanzania                                  87         0   
              2013 Tanzania                                   1         0   
              2015 Tanzania                                 137         0   
AG1000G-UG    2012 Uganda                                    82         0   
AG1000G-X    -1    Lab Cross                                  0        74   

taxon                                                gambiae  gcx1  gcx2  \
sample_set   year  country                                                 
AG1000G-AO    2009 Angola                                  0     0     0   
AG1000G-BF-A  2012 Burkina Faso                           98     0     0   
AG1000G-BF-B  2014 Burkina Faso                           46     0     0   
AG1000G-BF-C  2004 Burkina Faso                           13     0     0   
AG1000G-CD    2015 Democratic Republic of the Congo       76     0     0   
AG1000G-CF    1993 Central African Republic                2     0     0   
              1994 Central African Republic               53     0     0   
AG1000G-CI    2012 Cote d'Ivoire                           0     0     0   
AG1000G-CM-A  2009 Cameroon                              303     0     0   
AG1000G-CM-B  2005 Cameroon                               90     0     0   
AG1000G-CM-C  2013 Cameroon                               23     0     0   
AG1000G-FR    2011 Mayotte                                23     0     0   
AG1000G-GA-A  2000 Gabon                                  69     0     0   
AG1000G-GH

Here, we can see the breakdown by sample set for country, species and year. For the purposes of this notebook, let's use the Ghana sample set. If we wanted to use all sample sets, we could supply '3.0' instead of a sample set, which will load all samples from the ag3.0 release.

In [92]:
sample_set = 'AG1000G-GH'          # sample_set = '3.0' .you can also supply lists with multiple sample sets e.g ['AG1000G-GH', 'AG1000G-CI', 'AG1000G-BF-A]
sample_query = None # "aim_species == 'gambiae'"                # here we can subset to specific values in the metadata e.g :     "aim_species == 'gambiae'" , or "aim_species == 'arabiensis'"          

### **Plot allele frequencies in primers locations**

Now we can plot the primers pairs, and the frequency of any alternate alleles in the Ag1000g sample set of choice. When calculating allele frequencies, we will take the sum of all alternate alleles, as we are interested here in any mutations which are different from the reference genome. 

We will also plot the primer Tm, GC and genomic spans of each primer binding site. We can use this plot to identify primers pairs and probes which may be suitable, particularly trying to avoid SNPs in the 3' end. 

In [93]:
import allel

In [94]:
span_str=f'{contig}:{gdna_pos.min()}-{gdna_pos.max()}'                        

geno = ag3.snp_genotypes(region=span_str, sample_sets=sample_set, sample_query=sample_query) # get genotypes
ac = allel.GenotypeDaskArray(geno).count_alleles()

DEBUG:malariagen_data.ag3:snp_genotypes: normalise parameters
DEBUG:malariagen_data.ag3:snp_genotypes: normalise region to list to simplify concatenation logic
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate multiple sample sets and/or contigs
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_genotypes: locate region - do this only once, optimisation
DEBUG:malariagen_data.ag3:snp_sites: access SNP sites and concatenate over regions
DEBUG:malariagen_data.ag3:snp_sites: apply site mask if requested
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate data from multiple regions
DEBUG:malariagen_data.ag3:snp_genotypes: apply site filters if requested
DEBUG:malariagen_data.ag3:snp_genotypes: apply sample query if requested


In [95]:
alts = ag3.snp_sites(region=span_str, field='ALT').compute()

DEBUG:malariagen_data.ag3:snp_sites: access SNP sites and concatenate over regions
DEBUG:malariagen_data.ag3:snp_sites: apply site mask if requested


In [96]:
freqs = ac.to_frequencies() 
freqs = freqs.compute()

In [97]:
# if there are no ALTs lets add the zeros for the ALTs otherwise only REF count returned 

def addZeroCols(freqs):
  freqlength = freqs.shape[1]
  needed = 4-freqlength
  if needed > 0:
    for i in range(needed):
      freqs = np.column_stack([freqs, np.repeat(0, freqs.shape[0])])
  return(freqs)

In [99]:
alts = ag3.snp_sites(region=span_str, field='ALT')
refs = ag3.snp_sites(region=span_str, field='REF')
ref_alt_array = np.column_stack([refs, alts]).astype('U13')

DEBUG:malariagen_data.ag3:snp_sites: access SNP sites and concatenate over regions
DEBUG:malariagen_data.ag3:snp_sites: apply site mask if requested
DEBUG:malariagen_data.ag3:snp_sites: access SNP sites and concatenate over regions
DEBUG:malariagen_data.ag3:snp_sites: apply site mask if requested


In [100]:
import plotly
import plotly.express as px

In [101]:
freqs.shape

(2543, 4)

In [117]:
      snps = ag3.snp_calls(region=span_str, sample_sets=sample_set)     # get genotypes


DEBUG:malariagen_data.ag3:snp_calls: normalise parameters
DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class


In [245]:

def get_base_freqs(freqs, ref_alt_array):
  freq_df = pd.DataFrame(ref_alt_array)
  for i_base in range(4):
    for i in range(freqs.shape[0]):
      base = ref_alt_array[i, i_base]
      freq_df.loc[i, f'{base}_freq'] = freqs[i, i_base]
  return(freq_df)

def get_primer_arrays(contig, gdna_pos, sample_set, assay_type, sample_query=None):

  if any(item in assay_type for item in ['gDNA', 'probe']):
    span_str=f'{contig}:{gdna_pos.min()}-{gdna_pos.max()}'                        
    snps = ag3.snp_calls(region=span_str, sample_sets=sample_set)     # get genotypes
    ref_alt_arr = snps['variant_allele']
    geno = snps['call_genotype']
    freq_arr = allel.GenotypeArray(geno).count_alleles().to_frequencies() 
    pos_arr = gdna_pos
  elif assay_type == 'qPCR primers':
    freq_arr = []
    ref_alt_arr = []
    pos_arr = np.array([])
    exon_spans = np.array(AgamPrimer.consecutive(gdna_pos)) + 1
    for span in exon_spans:
      span_str = f'{contig}:{span[0]}-{span[1]}'                        
      snps = ag3.snp_calls(region=span_str, sample_sets=sample_set)     # get genotypes
      ref_alts = snps['variant_allele']
      geno = snps['call_genotype']
      freqs = allel.GenotypeArray(geno).count_alleles().to_frequencies()                                # calculate allele frequencies
      freqs = addZeroCols(freqs)
      freq_arr.append(freqs)
      ref_alt_arr.append(ref_alts)
      pos_arr = np.append(pos_arr, np.arange(span[0], span[1]+1).astype(int))
    freq_arr = np.concatenate(freq_arr)
    ref_alt_arr = np.concatenate(ref_alt_arr)

  return(freq_arr, ref_alt_arr.astype('U13'), pos_arr)


def get_primer_alt_frequencies(primer_df, gdna_pos, pair, sample_set, assay_type, contig, sample_query):
  """
  Find the genomic locations of pairs of primers, and runs span_to_freq
  to get allele frequencies at those locations
  """

  oligos, _ = AgamPrimer.return_oligo_list(assay_type)
  base_freqs, ref_alt_arr, pos_arr = get_primer_arrays(contig=contig, gdna_pos=gdna_pos, sample_set=sample_set, assay_type=assay_type, sample_query=sample_query)

  freq_arr = base_freqs[:, 1:].sum(axis=1)

  di = {}
  for oligo in oligos:
    primer_loc = primer_df.loc[f'PRIMER_{oligo}', str(pair)][0]
    primer_loc = primer_loc + 1 if oligo == 'RIGHT' else primer_loc
    primer_size = primer_df.loc[f'PRIMER_{oligo}', str(pair)][1]
    if oligo in ['LEFT', 'INTERNAL']:
      freq = freq_arr[primer_loc:primer_loc+primer_size]
      base_freqs_arr = base_freqs[primer_loc:primer_loc+primer_size, :]
      ref = ref_alt_arr[primer_loc:primer_loc+primer_size, 0]
      ref_alt = ref_alt_arr[primer_loc:primer_loc+primer_size, :]
      pos = pos_arr[primer_loc:primer_loc+primer_size]
    elif oligo == 'RIGHT':
      freq = np.flip(freq_arr[primer_loc-primer_size:primer_loc])
      base_freqs_arr = base_freqs[primer_loc-primer_size:primer_loc, :]
      base_freqs_arr = np.flip(base_freqs_arr, axis=0)
      ref = ref_alt_arr[primer_loc-primer_size:primer_loc, 0] 
      ref = np.array(list(AgamPrimer.rev_complement(''.join(ref))), dtype=str)
      ref_alt = ref_alt_arr[primer_loc-primer_size:primer_loc, :]
      pos = np.flip(pos_arr[primer_loc-primer_size:primer_loc])

    df = pd.DataFrame({'position': pos, 'base':ref, 'alt_frequency':freq}) # Make dataframe for plotting
    df['base_pos'] = df['base'] + "_" + df['position'].astype(str)
    assert df.shape[0] == primer_size, "Wrong size primers"
    print(oligo)
    freq_df = get_base_freqs(base_freqs_arr, ref_alt).filter(like='freq')
    df = pd.concat([df, freq_df], axis=1)
    di[oligo] = df
  return(di)

In [246]:
def plot_primer_ag3_frequencies(primer_df, gdna_pos, contig, sample_set, assay_type, seq_parameters, save=True, sample_query=None):
  """
  Loop through n primer pairs, retrieving frequency data and plot allele frequencies
  """

  if sample_query != None:
    print(f"Subsetting allele frequencies to {sample_query}")

  n_primer_pairs=len(primer_df.columns)
  name = seq_parameters['SEQUENCE_ID']
  exon_junctions = seq_parameters['SEQUENCE_OVERLAP_JUNCTION_LIST'] if assay_type == 'qPCR primers' else None
  transcript = seq_parameters['TRANSCRIPT'] if assay_type == 'qPCR primers' else None
  target_loc = seq_parameters['GENOMIC_SEQUENCE_TARGET'] if any(item in assay_type for item in ['gDNA', 'probe']) else None
  res_dict = {}
  # Loop through each primer pair and get the frequencies of alternate alleles, storing in dict
  for i in range(n_primer_pairs):
    res_dict[i] = get_primer_alt_frequencies(primer_df, gdna_pos, i, sample_set, assay_type, contig, sample_query)
  
  return(primer_df, res_dict, assay_type, exon_junctions, target_loc)

  # Plot data
  oligos, _ = return_oligo_list(assay_type)
  fig, ax = plt.subplots(n_primer_pairs, len(oligos), figsize=[6*len(oligos), (n_primer_pairs*2)+2], constrained_layout=True)    
  if any(item in assay_type for item in ['gDNA']):
    fig.suptitle(f"{name} primer pairs | {sample_set} | target {target_loc} bp", fontweight='bold')
  elif assay_type == 'probe':
    fig.suptitle(f"{name} probe | {sample_set} | target {target_loc} bp", fontweight='bold')
  elif assay_type == 'qPCR primers':
    fig.suptitle(f"{name} primer pairs | {sample_set} | target {transcript}", fontweight='bold')

  for i in range(n_primer_pairs):
    plot_primer(primer_df, ax, i, res_dict, assay_type, exon_junctions=exon_junctions, target_loc=target_loc)
  if save: fig.savefig(f"{name}.{assay_type}.primers.png")
  return(res_dict)


def plot_primer(primer_df, ax, i, res_dict, assay_type, exon_junctions=None, target_loc=None):
  """
  Plot primer allele frequencies and text
  """
  oligos, _ = AgamPrimer.return_oligo_list(assay_type)

  for idx, oligo in enumerate(oligos):
    if len(oligos) > 1:
      axes = ax[i, idx]
    else:
      axes = ax[i]

    sns.scatterplot(ax=axes, x=res_dict[i][oligo]['base_pos'], y=res_dict[i][oligo]['alt_frequency'], s=200) 
    axes.set_xticklabels(res_dict[i][oligo]['base'])
    axes.set_ylim(0,1)
    axes.set_xlabel("")
    axes.set_ylabel("Alternate allele frequency")
    if idx > 0: 
      axes.set_ylabel("")
      axes.set_yticklabels("")

    if assay_type == 'qPCR primers':
      a = primer_df.loc[f'PRIMER_{oligo}', str(i)]
      a = a[0] - a[1] if oligo == 'RIGHT' else a[0] + a[1]
      arr = np.array(exon_junctions) - a
      if (np.abs(arr) < 18).any():
        bases_in_new_exon  = arr[np.abs(arr) < 18]
        plt.setp(axes.get_xticklabels()[bases_in_new_exon[0]:], backgroundcolor="antiquewhite") if oligo == 'RIGHT' else plt.setp(axes.get_xticklabels()[:bases_in_new_exon[0]], backgroundcolor="antiquewhite")
    
    if oligo == 'LEFT':
      axes.set_title(f"Forward primer {i}") 
    elif oligo == 'RIGHT':
      axes.set_title(f"Reverse primer {i}")
    elif oligo == 'INTERNAL':
      axes.set_title(f"Probe {i}")
      idx = np.where(res_dict[i][oligo]['position'] == target_loc)[0][0]
      plt.setp(axes.get_xticklabels()[idx], backgroundcolor="antiquewhite")
    plot_pair_text(primer_df, i, axes, oligo, res_dict)

In [247]:

primer_df, res_dict, assay_type, exon_junctions, target_loc = plot_primer_ag3_frequencies(primer_df=primer_df,
                                                gdna_pos=gdna_pos,
                                                contig=contig,
                                                sample_set=sample_set, 
                                                sample_query=sample_query,
                                                assay_type=assay_type,
                                                seq_parameters=seq_parameters,
                                                save=True)

DEBUG:malariagen_data.ag3:snp_calls: normalise parameters
DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class


LEFT
RIGHT


DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class
DEBUG:malariagen_data.ag3:snp_calls: handle region, do thi

LEFT
RIGHT


DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class
DEBUG:malariagen_data.ag3:snp_calls: handle region, do thi

...

LEFT
RIGHT


DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class
DEBUG:malariagen_data.ag3:snp_calls: handle region, do thi

LEFT
RIGHT


DEBUG:malariagen_data.ag3:snp_calls: access SNP calls and concatenate multiple sample sets and/or regions
DEBUG:malariagen_data.ag3:_snp_calls_dataset: call arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: sample arrays
DEBUG:malariagen_data.ag3:_snp_calls_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_calls: add variants variables
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_position
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_allele
DEBUG:malariagen_data.ag3:_snp_variants_dataset: variant_contig
DEBUG:malariagen_data.ag3:_snp_variants_dataset: site filters arrays
DEBUG:malariagen_data.ag3:_snp_variants_dataset: set up attributes
DEBUG:malariagen_data.ag3:_snp_variants_dataset: create a dataset
DEBUG:malariagen_data.ag3:snp_calls: handle site class
DEBUG:malariagen_data.ag3:snp_calls: handle region, do thi

LEFT
RIGHT


In [248]:
res_dict[4]['RIGHT']

,position,base,alt_frequency,base_pos,C_freq,T_freq,A_freq,G_freq
0,28547471.0,A,0.0,A_28547471.0,1.0,0.0,0.0,0.0
1,28547470.0,C,0.0,C_28547470.0,0.0,1.0,0.0,0.0
2,28547469.0,A,0.0,A_28547469.0,1.0,0.0,0.0,0.0
3,28547468.0,A,0.0,A_28547468.0,0.0,0.0,1.0,0.0
4,28547467.0,G,0.0,G_28547467.0,1.0,0.0,0.0,0.0
5,28547466.0,A,0.0,A_28547466.0,0.0,1.0,0.0,0.0
6,28547465.0,T,0.0,T_28547465.0,0.0,0.0,0.0,1.0
7,28547464.0,C,0.0,C_28547464.0,1.0,0.0,0.0,0.0
8,28547463.0,G,0.0,G_28547463.0,1.0,0.0,0.0,0.0
9,28547462.0,T,0.0,T_28547462.0,1.0,0.0,0.0,0.0


In [267]:
lists = res_dict[i][oligo][['A_freq', 'C_freq', 'G_freq', 'T_freq']].to_numpy().T.tolist()

In [276]:
if len(oligos) == 2:
  plt_title = ['Forward primer', 'Reverse primer']
elif len(oligos) == 3:
  plt_title = ['Forward primer', 'Reverse primer', 'Probe']
  
title_list = []
for i in range(n_primer_pairs):
    for oligo in plt_title:
      title_list.append(f"{oligo} {i}")
from plotly.subplots import make_subplots
import plotly.graph_objects as go

name = 'vgsc'
oligos, _ = AgamPrimer.return_oligo_list(assay_type)

hover_template = "<br>".join(["Base / Position: %{customdata[4]}",
                                            "Total Alternate freq: %{y}",
                                            "A_freq: %{customdata[0]}",
                                            "C_freq: %{customdata[1]}",
                                            "G_freq: %{customdata[2]}",
                                            "T_freq: %{customdata[3]}"])
                             
fig = make_subplots(rows=n_primer_pairs, cols=len(oligos), subplot_titles=title_list, horizontal_spacing = 0.03, vertical_spacing=0.05)
for idx, oligo in enumerate(oligos):
  idx = idx+1
  for i in range(n_primer_pairs):
    row_i = i+1
    fig.add_trace(go.Scatter(x=res_dict[i][oligo]['base_pos'], 
                             y=res_dict[i][oligo]['alt_frequency'], customdata=res_dict[i][oligo][['A_freq', 'C_freq', 'G_freq', 'T_freq', 'base_pos']], 
                             hovertemplate=hover_template,
                             mode='markers', marker=dict(size=12, line=dict(width=2, color='black')), line=dict(color='dodgerblue'), marker_symbol='circle'), row=row_i, col=idx)
    fig.add_annotation(row=row_i, col=idx, x=res_dict[i][oligo]['base_pos'][0], y=0.8,
            text="5'",
            showarrow=False)
    fig.add_annotation(row=row_i, col=idx, x=res_dict[i][oligo]['base_pos'].to_numpy()[-1], y=0.8,
            text="3'",
            showarrow=False)
    
    fig.update_xaxes(row=row_i, col=idx, tickmode = 'array', tickvals = res_dict[i][oligo]['base_pos'], ticktext=res_dict[i][oligo]['base'], tickangle=0, mirror=True)
    if idx > 1:
      fig.update_yaxes(row=row_i, col=idx, range=[0,1], tickvals=np.arange(0, 1, 0.2), showticklabels=False, mirror=True)
    else:
      fig.update_yaxes(row=row_i, col=idx, tickvals=np.arange(0, 1, 0.2), range=[0,1], mirror=True)

    if ((row_i % 2) == 0) & (idx == 1):
      fig.update_yaxes(row=row_i, col=idx, title='Alternate allele frequency')

if any(item in assay_type for item in ['gDNA']):
  title_text = f"{name} primer pairs | {sample_set} | target {target_loc} bp"
elif assay_type == 'probe':
  title_text = f"{name} probe | {sample_set} | target {target_loc} bp"
elif assay_type == 'qPCR primers':
    title_text = f"{name} primer pairs | {sample_set} | target {transcript}"

#fig.update_traces(customdata=customdata, hovertemplate=hovertemplate)
fig.update_layout(height=1600, width=1600,
                    title_text=title_text, title_x = 0.5, template="simple_white", showlegend=False)

fig.show()

In [158]:
results_dict = AgamPrimer.plot_primer_ag3_frequencies(primer_df=primer_df,
                                                gdna_pos=gdna_pos,
                                                contig=contig,
                                                sample_set=sample_set, 
                                                sample_query=sample_query,
                                                assay_type=assay_type,
                                                seq_parameters=seq_parameters,
                                                save=True)

DEBUG:malariagen_data.ag3:snp_genotypes: normalise parameters
DEBUG:malariagen_data.ag3:snp_genotypes: normalise region to list to simplify concatenation logic
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate multiple sample sets and/or contigs
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate data from multiple sample sets
DEBUG:malariagen_data.ag3:snp_genotypes: locate region - do this only once, optimisation
DEBUG:malariagen_data.ag3:snp_sites: access SNP sites and concatenate over regions
DEBUG:malariagen_data.ag3:snp_sites: apply site mask if requested
DEBUG:malariagen_data.ag3:snp_genotypes: concatenate data from multiple regions
DEBUG:malariagen_data.ag3:snp_genotypes: apply site filters if requested
DEBUG:malariagen_data.ag3:snp_genotypes: apply sample query if requested
DEBUG:malariagen_data.ag3:snp_genotypes: normalise parameters
DEBUG:malariagen_data.ag3:snp_genotypes: normalise region to list to simplify concatenation logic
DEBUG:malariagen_data.ag3:snp_genotypes: co

KeyboardInterrupt: ignored

Now lets plot these primer pairs across the genome, highlighting where they are in relation to any nearby exons...

In [ ]:
AgamPrimer.plot_primer_locs(primer_res_dict=results_dict, assay_type=assay_type, gff=ag3.geneset(), contig=contig, seq_parameters=seq_parameters, n_primer_pairs=n_primer_pairs, legend_loc='lower left')

In [ ]:
primer_df

####**We may now have designed suitable primers. However, there are some further considerations...**


- Primers **must** be run in [**NCBI Blast**](https://blast.ncbi.nlm.nih.gov/Blast.cgi), to ensure specificity against the host organism, and specificity for the genomic location of interest. I would recommend both doing a general blast and also specifically against *An. gambiae* (TaxonID = 7165).

- If in multiplexed use with other primers or probes, primers must not interact with each other. This can be investigated on a one by one basis using the IDT tool [oligoanalyzer](https://eu.idtdna.com/calc/analyzer), though higher throughput algorithms may be required.

- If designing Locked Nucleic Acid (LNA) probes for SNP detection, you will want to play around with the placement of LNAs in the oligo sequence, which can allow short probes (~10-14 bases) to bind with high affinity and discriminate between SNPs. IDT have a tool for this which allow you to check the binding affinity between mismatches, though it requires a log in https://eu.idtdna.com/calc/analyzer/lna. 

- Many more considerations.... [IDT - How to design primers and probes for PCR and qPCR](https://eu.idtdna.com/pages/education/decoded/article/designing-pcr-primers-and-probes)   

\
  

####**Future development**

Any contributions or suggestions on how we can improve this notebook are more than welcome. Please [email](mailto:sanjay.nagi@lstmed.ac.uk) or log an [issue on github](https://github.com/sanjaynagi/primerDesignAg/issues). This notebook and source code for AgamPrimer are located here - https://github.com/sanjaynagi/AgamPrimer/    

\
####**References**

The Anopheles gambiae 1000 Genomes Consortium (2020). **Genome variation and population structure among 1142 mosquitoes of the African malaria vector species *Anopheles gambiae* and *Anopheles coluzzii***. *Genome Research*, **30**: 1533-1546. 
https://genome.cshlp.org/content/early/2020/09/25/gr.262790.120

Untergasser A, Cutcutache I, Koressaar T, Ye J, Faircloth BC, Remm M and Rozen SG (2012). **Primer3--new capabilities and interfaces**. *Nucleic Acids Research*. 40(15):e115.